# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import os

pth1 = 'data.csv'
pth2 = '/datasets/data.csv'
    
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [2]:
# Первые пять строк табицы данных
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df['total_income'].value_counts()

112874.418757    1
133912.272223    1
182036.676828    1
122421.963500    1
198271.837248    1
                ..
133299.194693    1
115080.782380    1
84896.781597     1
153838.839212    1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64

In [4]:
#Переименуем название столбца 'dob_years' на более понятный 'age'
df = df.rename(columns={'dob_years':'age'})

In [5]:
# Общая информация о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
#Общая статистика по столбцам с числовым типом данных
df.describe()

,children,days_employed,age,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [7]:
#Общая статистика по столбцам со строковым типом данных
df.describe(include='object')

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


In [8]:
#Уникальные значения по столбцам со строковым типом данных
display(df['education'].unique())
display(df['family_status'].unique())
display(df['gender'].unique())
display(df['income_type'].unique())
display(df['purpose'].unique())

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['F', 'M', 'XNA'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

Таблица данных содержит 12 столбцов и более 21 тыс. строк. Предварительно можно утверждать, что, данных достаточно для проверки гипотез. В каждой строке таблицы содержится информация о конкретном заемщике.  
Названия столбцов в целом соответствуют "хорошему стилю". В столбцах `days_employed` и `total_income` имеются пропуски.  
Из статистики по столбцам можно выделить следующие особенности набоа данных:  
* Медианное количество детей в семье составляет 0, т.е. заемщики в основной массе люди без детей. Минимальное значение количества детей в семье - (-1), т.е. отрицательное число, что свидетельствует о наличии некорректных значений в данном столбце. Максимальное количество детей в семье - 20, также требует внимания, т.к. значение слишком большое.
* Столбец с общим трудовым стажем в основном состоит из отрицательных значений, что наталкивает на мысль об ошибочности знака "минус" в данных. Однако имеются и положительные значения, причем максимальное значение столбца составляет 401 755 дней, что примерно равно 1 101 году. Таким образом, столбец содержит различные виды ошибок.
* Возраст заемщиков содержит нулевые значения.
* Месячный доход заемщиков изменяется от 20 до 226 тыс. у.е., среднее значение составляет - 167 тыс.у.е., медианное значение - 145 тыс. у.е. В столбце отсутствуют нулевые значения, однако имеются пропуски.
* Столбец с уровнем образования имеет 15 категорий, столбец с целями кредита насчитывает 38 категорий -  в данных столбцах имеется большое количество дублей. 
* Столбец с полом имеет 3 уникальных значения, из-за неявного пропуска вида 'XNA'.

## Шаг 2. Предобработка данных

### Обработка пропусков

На этапе изучения общей информации, определили, что явные пропуски содержатся в столбцах `days_employed` и `total_income`

In [9]:
#Определим количество пропусков в столбце 'days_employed'
df['days_employed'].isna().sum()

2174

In [10]:
#Количество пропусков в столбце 'total_income'
df['total_income'].isna().sum()

2174

Количество пропусков в столбцах с трудовым стажем и ежемесячным доходом совпадают. Можно предположить, что пропуски взаимосвязаны друг с другом, т.е. пропуск в столбце трудового стажа соответствует пропуску в столбце с ежемесячным доходом. Выясним, так ли это

In [11]:
(df['days_employed'].isna() == df['total_income'].isna()).value_counts()

True    21525
dtype: int64

Таким образом, стобцы `days_employed` и `total_income` полностью соответствуют друг другу по критерию наличия/отсутствия пропуска.  
Проверим, не связано ли наличие пропусков со значениями в других столбцах. Можно предположить, что пустые значения это нули, и они будут более характерными для некоторых типов занятости, например, безработных.

In [12]:
#Определим процент пропущенных значений для каждого типа занятости. 
df_income_type_all = df['income_type'].value_counts().rename('income_type_all')
df_income_type_nan = df[df['total_income'].isna()]['income_type'].value_counts().rename('income_type_nan')
df_income_type_merged = pd.merge(df_income_type_all, df_income_type_nan, how='left', left_index=True, right_index=True)
df_income_type_merged['percent_of_nan'] = (df_income_type_merged['income_type_nan'] /
                                         df_income_type_merged['income_type_all'] * 100).round(1)
df_income_type_merged

,income_type_all,income_type_nan,percent_of_nan
сотрудник,11119,1105.0,9.9
компаньон,5085,508.0,10.0
пенсионер,3856,413.0,10.7
госслужащий,1459,147.0,10.1
предприниматель,2,1.0,50.0
безработный,2,NaN,NaN
студент,1,NaN,NaN
в декрете,1,NaN,NaN


Гипотеза не подтвердилась, наличие пропусков не связано с типом занятости и составляет около 10% для каждой группы.  
Обратим также внимание, что некоторые типы занятости ("предприниматель", "безработный", "студент" и "в декрете") представлены единичными заемщиками.  
Связь пропусков с другими признаками менее правдоподобна, проверим для уровня образования клиентов и пола:

In [13]:
#Определим процент пропущенных значений для каждого уровня образования. 
df_education_all = df['education'].value_counts().rename('education_all')
df_education_nan = df[df['total_income'].isna()]['education'].value_counts().rename('education_nan')
df_education_merged = pd.merge(df_education_all, df_education_nan, how='left', left_index=True, right_index=True)
df_education_merged['percent_of_nan'] = (df_education_merged['education_nan'] /
                                         df_education_merged['education_all'] * 100).round(1)
df_education_merged

,education_all,education_nan,percent_of_nan
среднее,13750,1408.0,10.2
высшее,4718,496.0,10.5
СРЕДНЕЕ,772,67.0,8.7
Среднее,711,65.0,9.1
неоконченное высшее,668,55.0,8.2
ВЫСШЕЕ,274,23.0,8.4
Высшее,268,25.0,9.3
начальное,250,19.0,7.6
Неоконченное высшее,47,7.0,14.9
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29,7.0,24.1


In [14]:
#Определим процент пропущенных значений для каждого пола. 
df_gender_all = df['gender'].value_counts().rename('gender_all')
df_gender_nan = df[df['total_income'].isna()]['gender'].value_counts().rename('gender_nan')
df_gender_merged = pd.merge(df_gender_all, df_gender_nan, how='left', left_index=True, right_index=True)
df_gender_merged['percent_of_nan'] = (df_gender_merged['gender_nan'] /
                                         df_gender_merged['gender_all'] * 100).round(1)
df_gender_merged

,gender_all,gender_nan,percent_of_nan
F,14236,1484.0,10.4
M,7288,690.0,9.5
XNA,1,NaN,NaN


Похоже, что пропуски являются слуйчаными и не связаны со значениями в других столбцах.
Пропуски имеются в 2 174 строках, что составляет около 10% строк в датафрейме. Стратегия полного удаления пропусков в данном случае применима, но не желательна.  
Заполним пропуски типичными значениями. В случае ежемесячного дохода, предпочтительнее использовать медианное значение, т.к. среднее значение подвержено влияюнию выбросов.
Для определения наиболее типичного значения посмотрим на наличие связи между уровнем дохода и такими параметрами, как тип занятости, возраст клиента и уровень образования.

In [15]:
#Рассчитаем сгруппированные значения характеристик ежемесячного дохода по типу занятости
df.groupby('income_type')['total_income'].agg(['count','mean','median']).sort_values('median', ascending=False).round(1)

,count,mean,median
income_type,,,
предприниматель,1,499163.1,499163.1
компаньон,4577,202417.5,172358.0
госслужащий,1312,170898.3,150447.9
сотрудник,10014,161380.3,142594.4
безработный,2,131339.8,131339.8
пенсионер,3443,137127.5,118514.5
студент,1,98201.6,98201.6
в декрете,1,53829.1,53829.1


In [16]:
#Рассчитаем сгруппированные значения ежемесячного дохода по возрасту
df.groupby('age')['total_income'].agg(['count','mean','median']).sort_values('median', ascending=False).round(1)

,count,mean,median
age,,,
37,484,183334.9,160434.1
42,532,184711.0,159168.0
48,492,177231.2,158989.3
36,492,182428.9,158681.5
40,543,176353.7,158350.1
46,427,186505.6,157436.1
33,530,181604.7,156538.8
35,553,177914.0,156145.4
34,534,178092.1,155359.0


In [17]:
#Рассчитаем сгруппированные значения ежемесячного дохода по уровню образования
df.groupby('education')['total_income'].agg(['count','mean','median']).sort_values('median', ascending=False).round(1)

,count,mean,median
education,,,
УЧЕНАЯ СТЕПЕНЬ,1,198570.8,198570.8
ученая степень,4,187794.4,185687.1
Высшее,243,215199.7,180720.7
высшее,4222,207108.3,175625.3
ВЫСШЕЕ,251,199917.9,167242.2
НЕОКОНЧЕННОЕ ВЫСШЕЕ,22,176496.6,163911.6
Неоконченное высшее,40,159188.4,160402.5
неоконченное высшее,613,183172.9,159780.9
среднее,12342,153853.3,136643.3


Видно, что существует взаимосвязь уровня дохода со всеми признаками, которые мы рассмотрели. Типичное значение можно определить используя медианное значение столбца в целом, однако правильнее это будет сделать на основе какого-либо из рассмотренных параметров или их комбинации (при этом при использовании возраста необходимо провести категоризацию признака и выделить укрупненные диапазоны, при использовании уровня образования необходимо обработать дубли). Для простоты используем тип занятости.

In [18]:
#Сгруппируем данные по типовым значениям дохода для различных типов занятости
df_group_income = df.groupby('income_type')['total_income'].median().rename('total_income_median')
df_group_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income_median, dtype: float64

In [19]:
#Удалим строку с пропуском по типу занятости 'предприниматель',
#т.к. данный тип единичен и не может быть охарактеризован типичным значением
df = df.drop(df[(df['income_type'] == 'предприниматель') & (df['total_income'].isna())].index)

In [20]:
#Создадим столбец 'total_income_median' в исходном датафрейме с типовыми значениями 
df = df.merge(df_group_income, how='left', on = 'income_type')
#заполним данными из столбца 'total_income_median' пустые значения столбца 'total_income'
df['total_income'].fillna(df['total_income_median'], inplace=True)
#и удалим столбец 'total_income_median', т.к. он больше не нужен
df.drop('total_income_median', axis=1, inplace=True)

In [21]:
#Проверим отсутствие пропусков в столбце'total_income'
df['total_income'].isna().sum()

0

На этапе изучения общей информации мы определили, что столбец `days_employed` содержит отрицательные значения трудового стажа, а также нереалистичные значения, превышающие 1000 лет. Проанализируем столбец подробнее

In [22]:
#Количество отрицательных значений
df[df['days_employed'] < 0]['days_employed'].count()

15906

In [23]:
#Статистика по положительным значениям
df[df['days_employed'] > 0]['days_employed'].describe()

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

Таким образом, большая часть столбца `days_employed` заполнена отрицательными значениями, а положительные значения трудового стажа находятся в интервале от 900 до 1100 лет. Данный столбец лучше удалить из наших данных

In [24]:
df.drop('days_employed', axis=1, inplace=True)

In [25]:
#Проверим отсутствие пропусков в очищенном датафрейме
df.isna().sum()

children            0
age                 0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проанализируем и обработаем остальные обнаруженные нами неявные пропуски и некорректные значения.  
Начнем со столбца `children`

In [26]:
df['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Столбец с количеством детей содержит 47 строк со значением (-1). Возможно ошибочно проставлен знак "минус". Однако выяснить это мы не можем, а процент отрицательных значений составляет около  0,2% от общего количества строк, поэтому удаление этих строк будет приемлимым решением в данном случае.  
Обращает внимание также факт наличия в таблице данных о семьях с 20 детьми. Это вероятно также ошибочные данные, семьи с 20 детьми были бы уникальными, единичными случаями, а в нашем датафрейми их количество - 76, что превышает количество семей с четырьмя и пятью детьми. При этом в таблице нет данных о семьях с количеством детей от 6 до 19. Возможно таким образом были сгруппированы семьи с количеством детей более пяти или ошибочно появился лишний ноль после значения 2. В документации информации об этом нет, возможность уточнить также отсутствует. Примем решение об удалении этих строк. Процент таких строк составялет около 0,3% от общего количества.

In [27]:
df = df.drop(df[(df['children'] == -1) | (df['children'] == 20)].index)

In [28]:
df['children'].value_counts()

0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

При изучении общей информации по данным было замечено, что возраст заемщиков содержит нулевые значения. Проанализируем столбец `age` подробнее

In [29]:
df['age'].value_counts().sort_index()

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    460
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: age, dtype: int64

В столбце `dob_years` содержится 100 нулевых значения, других выбивающихся значений в столбце нет. Количество строк с нулевым возрастом сравнительно небольшое по отношению к объему датасета, вероятно это отсутствующие значения заполнены таким образом, удалим эти строки, т.к. возраст представляется важным прогностическим признаком и заполнение его типичными значениями может привести к снижению точности модели кредитного скорринга.

In [30]:
df = df.drop(df[df['age'] == 0].index)

In [31]:
df[df['age'] == 0]['age'].count()

0

Рассмотрим столбец с полом клиентов

In [32]:
df['gender'].value_counts()

F      14083
M       7217
XNA        1
Name: gender, dtype: int64

Имеется всего один пропуск в столбце `gender`, удалим эту строку.

In [33]:
df = df.drop(df[df['gender'] == 'XNA'].index)

In [34]:
df['gender'].value_counts()

F    14083
M     7217
Name: gender, dtype: int64

In [35]:
#Общая информация по датафрейму после обработки всех пропусков и некорректных значений
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21300 entries, 0 to 21523
Data columns (total 11 columns):
children            21300 non-null int64
age                 21300 non-null int64
education           21300 non-null object
education_id        21300 non-null int64
family_status       21300 non-null object
family_status_id    21300 non-null int64
gender              21300 non-null object
income_type         21300 non-null object
debt                21300 non-null int64
total_income        21300 non-null float64
purpose             21300 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В ходе этапа по обработке пропусков были выполнены следующие операции очистки данных:  
 * пропуски в столбце с ежемесячным доходом заполнены типичными (медианными) значениями в зависимости от типа занятости клиента,
 * столбец с трудовым стажем полностью удален из датасета вследствие наличия разнотипных ошибок и практически полной непригодности для обработки,
 * удалены строки содержащие ошибочные данные в столбцах с количеством детей, полом и возрастом.  

Количество строк в очищенном датасте составляет 21 300, что на 225 строк меньше изначальной таблицы, т.е. был удален примерно 1% данных.

### Замена типа данных

In [36]:
# Типы данных по столбцам
df.dtypes

children              int64
age                   int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

В данных остался один столбец `total_income` с вещественным типом данных, потребности в высокой точности и сохранения дробной части числа у нас нет, для удобства преобразуем столбец в целичисленный тип.

In [37]:
df['total_income'] = df['total_income'].astype(int)
df['total_income'].dtypes

dtype('int32')

**Вывод**

Тип данных столбца с ежемесячным доходом заемщиков преобразован с вещественного на целочисленный.

### Обработка дубликатов

Определим количество и удалим строки с полными дублями в данных

In [38]:
#Количество полных дублей
df.duplicated().sum()

54

In [39]:
#Удалим дублирующиеся строки
df.drop_duplicates(inplace=True)

In [40]:
df.duplicated().sum()

0

На этапе рассмотрения общей информации было отмечено наличие неявных дубликатов в столбцах `education` и `purpose`

In [41]:
#Уникальные значения столбца 'education'
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Появление дубликатов связано с использованием разного регистра. Для обработки дубликатов в данном столбце приведем все символы к нижнему регистру

In [42]:
df['education'] = df['education'].str.lower()

In [43]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [44]:
#Уникальные значения столбца 'purpose'
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Данный случай сложнее. Дубликаты связаны с указанием одинаковых или похожих целей с использования разных словоформ и синонимичных выражений. Вероятно клиенты заполняли информацию о целях кредита в свободной форме

**Вывод**

На данном этапе  были выялены и удалены полностью дублирующиеся строки, а также обработаны неявные дубли в столбце `education`, связанные с заполнением данных с использованием разного регистра символов.

### Лемматизация

In [45]:
from pymystem3 import Mystem
m = Mystem()

In [46]:
#Создадим список из уникальных значений столбца 'purpose'
purpose_list = list(df['purpose'].unique())
purpose_list

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [47]:
#Создадим словарь, ключами которого будут уникальные значению 'purpose',
#а значениями - списки из соответствующих им лемм
lemma_dict = {}
for elem in purpose_list:
    lemma_dict.update({elem:m.lemmatize(elem)})

lemma_dict

{'покупка жилья': ['покупка', ' ', 'жилье', '\n'],
 'приобретение автомобиля': ['приобретение', ' ', 'автомобиль', '\n'],
 'дополнительное образование': ['дополнительный', ' ', 'образование', '\n'],
 'сыграть свадьбу': ['сыграть', ' ', 'свадьба', '\n'],
 'операции с жильем': ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 'образование': ['образование', '\n'],
 'на проведение свадьбы': ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 'покупка жилья для семьи': ['покупка',
  ' ',
  'жилье',
  ' ',
  'для',
  ' ',
  'семья',
  '\n'],
 'покупка недвижимости': ['покупка', ' ', 'недвижимость', '\n'],
 'покупка коммерческой недвижимости': ['покупка',
  ' ',
  'коммерческий',
  ' ',
  'недвижимость',
  '\n'],
 'покупка жилой недвижимости': ['покупка',
  ' ',
  'жилой',
  ' ',
  'недвижимость',
  '\n'],
 'строительство собственной недвижимости': ['строительство',
  ' ',
  'собственный',
  ' ',
  'недвижимость',
  '\n'],
 'недвижимость': ['недвижимость', '\n'],
 'строительство недвижимости': ['стр

In [48]:
#Удалим из значений словаря лишние символы и предлоги для облегчения визуального восприятия
new_lemma_dict ={}
for key,value in lemma_dict.items():
    new_value_list = []
    for elem in value:
        if elem not in [' ', '\n', 'с', 'для', 'на', 'со']:
            new_value_list.append(elem)
    new_lemma_dict.update({key:new_value_list})
new_lemma_dict

{'покупка жилья': ['покупка', 'жилье'],
 'приобретение автомобиля': ['приобретение', 'автомобиль'],
 'дополнительное образование': ['дополнительный', 'образование'],
 'сыграть свадьбу': ['сыграть', 'свадьба'],
 'операции с жильем': ['операция', 'жилье'],
 'образование': ['образование'],
 'на проведение свадьбы': ['проведение', 'свадьба'],
 'покупка жилья для семьи': ['покупка', 'жилье', 'семья'],
 'покупка недвижимости': ['покупка', 'недвижимость'],
 'покупка коммерческой недвижимости': ['покупка',
  'коммерческий',
  'недвижимость'],
 'покупка жилой недвижимости': ['покупка', 'жилой', 'недвижимость'],
 'строительство собственной недвижимости': ['строительство',
  'собственный',
  'недвижимость'],
 'недвижимость': ['недвижимость'],
 'строительство недвижимости': ['строительство', 'недвижимость'],
 'на покупку подержанного автомобиля': ['покупка', 'подержать', 'автомобиль'],
 'на покупку своего автомобиля': ['покупка', 'свой', 'автомобиль'],
 'операции с коммерческой недвижимостью': ['о

Каждое значение полученного словаря содержит одно из приведенных ниже основных слов:  
* жилье
* автомобиль
* образование
* свадьба
* недвижимость  
 
Таким образом, любая из 38 целей из столбца `purpose` может быть определена в одну из пяти укрупненных категорий.  
Дополнительно необходимо будет объединить цели из категорий `недвижимость` и `жилье`, т.к. это синонимы. В итоге останется четыре укрупненные категории в столбце с целями заема.

In [49]:
#Поставим в соответствие каждой цели из датасета, соответствующую цель из списка типичных целей
purpose_list = ['жилье','автомобиль','образование','свадьба','недвижимость']
purpose_dict ={}
for key,value in new_lemma_dict.items():
    for elem in value:
        if elem in purpose_list :
            purpose_dict.update({key:elem})
            break
purpose_dict

{'покупка жилья': 'жилье',
 'приобретение автомобиля': 'автомобиль',
 'дополнительное образование': 'образование',
 'сыграть свадьбу': 'свадьба',
 'операции с жильем': 'жилье',
 'образование': 'образование',
 'на проведение свадьбы': 'свадьба',
 'покупка жилья для семьи': 'жилье',
 'покупка недвижимости': 'недвижимость',
 'покупка коммерческой недвижимости': 'недвижимость',
 'покупка жилой недвижимости': 'недвижимость',
 'строительство собственной недвижимости': 'недвижимость',
 'недвижимость': 'недвижимость',
 'строительство недвижимости': 'недвижимость',
 'на покупку подержанного автомобиля': 'автомобиль',
 'на покупку своего автомобиля': 'автомобиль',
 'операции с коммерческой недвижимостью': 'недвижимость',
 'строительство жилой недвижимости': 'недвижимость',
 'жилье': 'жилье',
 'операции со своей недвижимостью': 'недвижимость',
 'автомобили': 'автомобиль',
 'заняться образованием': 'образование',
 'сделка с подержанным автомобилем': 'автомобиль',
 'получение образования': 'образов

In [50]:
#Функция для замены названий целей
def replace_purpose(row):
    '''
    Функция принимает на вход старое название цели,
    в цикле перебирает элементы словаря purpose_dict,
    если находит старый вариант написания целей - ключ словаря,
    возвращает новый вариант написания - значение словаря
    '''
    for key,value in purpose_dict.items():
        if key == row:
            return value
    else:
        return('Внимание - неопознанная цель')

In [51]:
#Протестируем функцию replace_purpose
row_test_values=['получение образования', 'ремонт жилью', 'сыграть свадьбу', 'бухнуть']
for row in row_test_values:
    print(replace_purpose(row))


образование
жилье
свадьба
Внимание - неопознанная цель


In [52]:
#Создадим столбец с обновленными названиями целей
df['purpose_type'] = df['purpose'].apply(replace_purpose)
df.head()

,children,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [53]:
#Заменим цель "жилье" на "недвижимость"
df.loc[df['purpose_type'] == 'жилье', ['purpose_type']] = 'недвижимость' 
df.head()

,children,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [54]:
df['purpose_type'].value_counts()

недвижимость    10705
автомобиль       4260
образование      3971
свадьба          2310
Name: purpose_type, dtype: int64

**Вывод**

Анализ столбца `purpose` показал, что большое количество уникальных значений в данном столбце связано с использованием для его заполнения различных словоформ и синонимичных выражений, обозначающих одинаковые или схожие цели.  
Для обработки таких дублей и распределения их по укрупненным категориям использован метод лемматизации. Были вычленены основные слова, характеризующие цель заема, на которые были заменены значения столбца. В итоге 38 изначальных выражения были объединены в четыре категории: недвижимость, автомобиль, образование и свадьба.


### Категоризация данных

Для категоризации рассмотрим такие признаки как количество детей, возраст, уровень образования, семейное положение, ежемесячный доход

In [55]:
#Количество уникальных значений по столбцу `children`
df['children'].value_counts()

0    14036
1     4793
2     2039
3      328
4       41
5        9
Name: children, dtype: int64

Из распределения видно, что в выборке сравнительно мало семей с четырьмя и пятью детьми. Их количество может оказаться недостаточным для формулировки выводов.  
Представляется логичным объединение семей с 3, 4 и 5 детьми в одну группу - более 2-х детей.

In [56]:
def children_group(children):
    '''
    Функция возвращает группу по количеству детей children:
    - 'нет детей', если children = 0;
    - '1 ребенок', если children = 1;
    - '2 детей', если children = 2;
    - 'более 2-х детей', если children > 2;
    '''    
    if children == 0:
        return 'нет детей'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 детей'
    return 'более 2-х детей'


In [57]:
df['children_group'] = df['children'].apply(children_group)
df['children_group'].value_counts()

нет детей          14036
1 ребенок           4793
2 детей             2039
более 2-х детей      378
Name: children_group, dtype: int64

Группа с количеством детей более 2-х сравнительно малочисленна, однако еще большее укрупнение групп может негативно сказаться на качестве модели.

In [58]:
#Количество уникальных значений по столбцу `age`
df['age'].value_counts()

35    613
41    602
40    601
34    595
38    594
42    591
33    577
39    571
31    555
36    552
29    542
44    541
48    535
30    534
37    530
43    509
50    508
49    505
32    505
28    501
45    494
27    490
52    483
56    479
47    477
54    473
46    467
53    457
58    455
57    453
51    446
55    441
59    440
26    406
60    373
25    356
61    352
62    348
63    268
24    262
64    260
23    251
65    194
22    183
66    182
67    167
21    110
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: age, dtype: int64

In [59]:
#Функция для группировки заемщиков по возрасту
def age_group(age):
    if age < 25:
        return 'до 25 лет'
    if 25 <= age < 35:
        return 'от 25 до 35 лет'
    if 35 <= age < 45:
        return 'от 35 до 45 лет'
    if 45 <= age < 55:
        return 'от 45 до 55 лет'
    if 55 <= age < 65:
        return 'от 55 до 65 лет'
    return 'более 65 лет'

In [60]:
df['age_group'] = df['age'].apply(age_group)
df['age_group'].value_counts()

от 35 до 45 лет    5704
от 25 до 35 лет    5061
от 45 до 55 лет    4845
от 55 до 65 лет    3869
более 65 лет        896
до 25 лет           871
Name: age_group, dtype: int64

Каждая группа по возрасту получилась достаточно представительной, такую категоризацию можно оставить для использования в модели.  
Т.к. названия категорий по возрасту получились достаточно грмоздкие, создадим столбец `age_group_id` и каждой возрастной группе присвоим соответствующий номер.

In [61]:
#Функция для присвоения возрастной группе соответствующего номера
def age_group_id(age_group):
    if age_group == 'до 25 лет':
        return 0
    if age_group == 'от 25 до 35 лет':
        return 1
    if age_group == 'от 35 до 45 лет':
        return 2
    if age_group == 'от 45 до 55 лет':
        return 3
    if age_group == 'от 55 до 65 лет':
        return 4
    return 5

In [62]:
df['age_group_id'] = df['age_group'].apply(age_group_id)

In [63]:
#Создадим "словарь" соответствия age_group - age_group_id
age_group_dict = df[['age_group', 'age_group_id']].drop_duplicates().sort_values('age_group_id').reset_index(drop=True)
age_group_dict

,age_group,age_group_id
0,до 25 лет,0
1,от 25 до 35 лет,1
2,от 35 до 45 лет,2
3,от 45 до 55 лет,3
4,от 55 до 65 лет,4
5,более 65 лет,5


In [64]:
#Распределение заемщиков по уровню образования
df['education'].value_counts()

среднее                15028
высшее                  5192
неоконченное высшее      738
начальное                282
ученая степень             6
Name: education, dtype: int64

In [65]:
#Проверим соответствие education и education_id
df.groupby('education')['education_id'].unique().sort_values()

education
высшее                 [0]
среднее                [1]
неоконченное высшее    [2]
начальное              [3]
ученая степень         [4]
Name: education_id, dtype: object

Количество заемщиков с ученой степенью слишком мало, всего шесть человек. Укрупним категорию с высшим образованием, включив в нее также группу с ученой степенью. Произведем замену в столбце `education` без добавления нового столбца.  
Индексация `education_id` не является интуитивно понятной: 0 соответсвутет высшему образованию, 1 - среднему, 2 - неоконченному высшему и т.д. Перезапишем значения таким образом, чтобы более высокий уровень образования соответствовал более высокому значению `education_id`

In [66]:
df.loc[df['education'] == 'ученая степень', 'education'] = 'высшее'
df['education'].value_counts()

среднее                15028
высшее                  5198
неоконченное высшее      738
начальное                282
Name: education, dtype: int64

In [67]:
df.loc[df['education'] == 'начальное', 'education_id'] = 0
df.loc[df['education'] == 'среднее', 'education_id'] = 1
df.loc[df['education'] == 'неоконченное высшее', 'education_id'] = 2
df.loc[df['education'] == 'высшее', 'education_id'] = 3
df.groupby('education')['education_id'].unique().sort_values()

education
начальное              [0]
среднее                [1]
неоконченное высшее    [2]
высшее                 [3]
Name: education_id, dtype: object

In [68]:
#Распределение заемщиков по семейному положению
df['family_status'].value_counts()

женат / замужем          12217
гражданский брак          4124
Не женат / не замужем     2780
в разводе                 1179
вдовец / вдова             946
Name: family_status, dtype: int64

In [69]:
#Проверим соответствие семейного статуса и соответствующего номера id
df.groupby('family_status')['family_status_id'].unique().sort_values()

family_status
женат / замужем          [0]
гражданский брак         [1]
вдовец / вдова           [2]
в разводе                [3]
Не женат / не замужем    [4]
Name: family_status_id, dtype: object

В каждой группе по семейному положению достаточно большое количество клиентов, групп сравнительно немного - всего пять, оставим все как есть

In [70]:
#Распределение заемщиков по типу занятости
df['income_type'].value_counts()

сотрудник          10968
компаньон           5028
пенсионер           3800
госслужащий         1445
безработный            2
предприниматель        1
студент                1
в декрете              1
Name: income_type, dtype: int64

Категории "безработный", "студент", "в декрете", "предприниматель" представлены в единичном количестве.
Требует уточнения понятие "компаньон", т.к. оно не характерно для российской практики, вероятнее всего под этим понятием имеются ввиду владельцы/совладельцы компаний. Тогда с небольшой натяжкой можно отнести единственного "предпринимателя" в эту категорию, а остальные категории удалим, т.к. из них не получится собрать представительную выборку даже если объединить в одну категорию, например 'прочие'.

In [71]:
df = df.drop(df[(df['income_type'] == 'безработный') | (df['income_type'] == 'студент') |
                (df['income_type'] == 'в декрете')].index)
df.loc[df['income_type'] == 'предприниматель', 'income_type'] = "компаньон"

In [72]:
df['income_type'].value_counts()

сотрудник      10968
компаньон       5029
пенсионер       3800
госслужащий     1445
Name: income_type, dtype: int64

In [73]:
#Статистика заемщиков по ежемесячному доходу
df['total_income'].describe().round(1)

count      21242.0
mean      165355.3
std        98368.7
min        20667.0
25%       107644.8
50%       142594.0
75%       195750.2
max      2265604.0
Name: total_income, dtype: float64

Категоризацию можно произвести на основе вышеприведенных квартилей, в этом случае будет всего 4 категории с одинаковым количеством заемщиков внутри каждой группы. Этого может оказаться недостаточным для построения модели, поэтому увеличим количество категорий до 6 и определим примерно их границы

In [74]:
pd.qcut(df['total_income'], q=6).unique().sort_values()

[(20666.999, 92176.333], (92176.333, 119257.333], (119257.333, 142594.0], (142594.0, 172357.0], (172357.0, 228839.0], (228839.0, 2265604.0]]
Categories (6, interval[float64]): [(20666.999, 92176.333] < (92176.333, 119257.333] < (119257.333, 142594.0] < (142594.0, 172357.0] < (172357.0, 228839.0] < (228839.0, 2265604.0]]

In [75]:
#Функция для группировки заемщиков по уровню дохода
def income_group(total_income):
    '''
    Функция принимает значение total_income,
    возвращает соответствующее название группы
    и соответстующий данной группе номер (id)
    '''
    if total_income < 90000:
        return pd.Series(['до 90 000', '0'])
    if 90000 <= total_income < 120000:
        return pd.Series(['от 90 000 до 120 000', '1'])
    if 120000 <= total_income < 150000:
        return pd.Series(['от 120 000 до 150 000', '2'])
    if 150000 <= total_income < 180000:
        return pd.Series(['от 150 000 до 180 000', '3'])
    if 180000 <= total_income < 230000:
        return pd.Series(['от 180 000 до 230 000', '4'])
    return pd.Series(['более 230 000', '5'])


In [76]:
df[['income_group', 'income_group_id']] = df['total_income'].apply(income_group)
df.head()

,children,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,children_group,age_group,age_group_id,income_group,income_group_id
0,1,42,высшее,3,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1 ребенок,от 35 до 45 лет,2,более 230 000,5
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребенок,от 35 до 45 лет,2,от 90 000 до 120 000,1
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,от 25 до 35 лет,1,от 120 000 до 150 000,2
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,более 2-х детей,от 25 до 35 лет,1,более 230 000,5
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,от 45 до 55 лет,3,от 150 000 до 180 000,3


In [77]:
#Распределение количества заемщиков по категориям
df['income_group'].value_counts()

от 120 000 до 150 000    4352
от 90 000 до 120 000     3847
более 230 000            3479
от 150 000 до 180 000    3375
до 90 000                3314
от 180 000 до 230 000    2875
Name: income_group, dtype: int64

In [78]:
#"Cловарь" соответствия income_group - income_group_id
income_group_dict = df[['income_group', 'income_group_id']].drop_duplicates().sort_values('income_group_id').reset_index(drop=True)
income_group_dict

,income_group,income_group_id
0,до 90 000,0
1,от 90 000 до 120 000,1
2,от 120 000 до 150 000,2
3,от 150 000 до 180 000,3
4,от 180 000 до 230 000,4
5,более 230 000,5


Удалим из таблицы лишние столбцы до категоризации и длинные названия категорий, оставив только соответствующие им номера

In [79]:
df_short = df.drop(['children', 'age', 'age_group', 'total_income', 'purpose', 'income_group'], axis=1)

In [80]:
df_short.head()

,education,education_id,family_status,family_status_id,gender,income_type,debt,purpose_type,children_group,age_group_id,income_group_id
0,высшее,3,женат / замужем,0,F,сотрудник,0,недвижимость,1 ребенок,2,5
1,среднее,1,женат / замужем,0,F,сотрудник,0,автомобиль,1 ребенок,2,1
2,среднее,1,женат / замужем,0,M,сотрудник,0,недвижимость,нет детей,1,2
3,среднее,1,женат / замужем,0,M,сотрудник,0,образование,более 2-х детей,1,5
4,среднее,1,гражданский брак,1,F,пенсионер,0,свадьба,нет детей,3,3


**Вывод**

Была произведена категоризация числовых признаков: количество детей, возраст заемщиков, ежемесячный доход. Некоторым из  категорий (возраст заемщиков, ежемесячный доход), характеризующимися длинными названиями, присвоен номер для удобства последющей работы с ними, сформированы "словари" соответствия.   
Укрупненны категории и преобразовыны идентификаторы по уровню образования клиентов, некоторые категории по типу занятости, представленные единичными данными, были исключены из выборки.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [81]:
#Построим сводную таблицу между категориями клиентов по количеству детей и возращаемостью кредита
pivot_children =df_short.pivot_table(index='children_group', columns='debt', values='gender', aggfunc='count', margins=True)

In [82]:
pivot_children

debt,0,1,All
children_group,,,
1 ребенок,4352,440,4792
2 детей,1845,193,2038
более 2-х детей,347,31,378
нет детей,12976,1058,14034
All,19520,1722,21242


In [83]:
#Определим процент невозвратов для каждой категории по семейному положению
pivot_children['percent_non_repayment'] = (pivot_children[1] / pivot_children['All'] * 100).round(1)
pivot_children.sort_values('percent_non_repayment')

debt,0,1,All,percent_non_repayment
children_group,,,,
нет детей,12976,1058,14034,7.5
All,19520,1722,21242,8.1
более 2-х детей,347,31,378,8.2
1 ребенок,4352,440,4792,9.2
2 детей,1845,193,2038,9.5


**Вывод**

Лучше всего с возвращаемостью обстоит дело у клиентов без детей (процент невозвратов 7,5%), хуже всего у клиентов с 2мя детьми (9,5%) и с 1 ребенком (9,2%). При количестве детей более двух  - процент невозвратов составляет 8,1%, т.е. лучше чем у клиентов с одним или двумя детьми, но хуже чем у клиентов без детей. Это может быть связано с тем, что больше двух детей заводят в основном более состоятельные и соответственно, кредитоспособные, люди. Также причина может быть связана с малым объемом выборки для данной категории, и соответственно, ее возможной нерепрезентативностью.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [84]:
pivot_family_status =df_short.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
pivot_family_status['percent_non_repayment'] = (pivot_family_status[1] / (pivot_family_status[0] +
                                                                          pivot_family_status[1]) * 100).round(1)
pivot_family_status.sort_values('percent_non_repayment')

debt,0,1,percent_non_repayment
family_status,,,
вдовец / вдова,884,62,6.6
в разводе,1095,84,7.1
женат / замужем,11294,921,7.5
гражданский брак,3740,383,9.3
Не женат / не замужем,2507,272,9.8


**Вывод**

Лучше всего с возвращаемостью кредитов в срок обстоят дела у вдов/вдовцов (невозврат - 6,6%) и клиентов, находящихся в разводе (невозврат - 7,1%). Больше всего проблем с возвратом кредитов у клиентов не состоявших в браке (9,8 %) и клиентов, состоящих в гражданском браке (9,3%). Вероятно такая связь кореллирует с возрастом, логично предположить, что вдовы/вдовцы это наиболее пожилые клиенты; клиенты, находящиеся в разводе - скорее всего из категории пожилых и средних лет, уже имеющие опыт семейной жизни и успевшие развестись; клиенты находящиеся в гражданском браке - вероятно, в основной массе, молодые люди и люди средних лет, еще не успевшие зарегестрировать отношения; а клиенты никогда не состоявшие в браке - это наиболее молодая категория клиентов.  
Основная категория клиентов, состоит в официальных отношениях и имеет процент невозвратов на уровне 7,5%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [85]:
pivot_age_group_id =df_short.pivot_table(index='age_group_id', columns='debt', values='gender', aggfunc='count')
pivot_age_group_id ['percent_non_repayment'] = (pivot_age_group_id[1] / (pivot_age_group_id[0] +
                                                                          pivot_age_group_id[1]) * 100).round(1)
pivot_age_group_id.sort_values('percent_non_repayment')

debt,0,1,percent_non_repayment
age_group_id,,,
4,3656,213,5.5
5,847,49,5.5
3,4502,342,7.1
2,5232,471,8.3
0,782,88,10.1
1,4501,559,11.0


**Вывод**

Как видим, существует достаточно четкая связь между возрастом и возвращаемостью кредитов. Наилучшая возвращаемость у клиентов из возрастных групп 4 и 5 - по 5,5% невозврата (категория клиентов старше 55 лет). Больше всего невозвратов у наиболее молодых клиентов из категории 0 (10,1%) и 1 (22%) - это клиенты моложе 35 лет.  
Проверим также гипотезу о связи между возрастом и семейным положением, высказанный при проверке связи между возвращаемостью клиентов и семейным положением

In [86]:
#Сводня таблица общего количества клиентов в разрезе семейного положения и возрастной группы
pivot_age_family =df_short.pivot_table(index='family_status', columns='age_group_id', values='debt', 
                                       aggfunc='count', margins=True)
pivot_age_family

age_group_id,0,1,2,3,4,5,All
family_status,,,,,,,
Не женат / не замужем,360,950,575,465,344,85,2779
в разводе,15,183,344,338,250,49,1179
вдовец / вдова,1,16,81,208,471,169,946
гражданский брак,220,1032,1179,904,645,143,4123
женат / замужем,274,2879,3524,2929,2159,450,12215
All,870,5060,5703,4844,3869,896,21242


In [87]:
#Процентное соотношение количества клиентов по возрастным группам 
#в каждой категории по семейному положению
group_family = df_short.pivot_table(index='family_status', values='debt', aggfunc='count', margins=True)
pivot_age_family_percent = pivot_age_family.div(group_family['debt']*0.01, axis=0).round(1)
pivot_age_family_percent

age_group_id,0,1,2,3,4,5,All
family_status,,,,,,,
Не женат / не замужем,13.0,34.2,20.7,16.7,12.4,3.1,100.0
в разводе,1.3,15.5,29.2,28.7,21.2,4.2,100.0
вдовец / вдова,0.1,1.7,8.6,22.0,49.8,17.9,100.0
гражданский брак,5.3,25.0,28.6,21.9,15.6,3.5,100.0
женат / замужем,2.2,23.6,28.8,24.0,17.7,3.7,100.0
All,4.1,23.8,26.8,22.8,18.2,4.2,100.0


Действительно, основное количество вдов/вдовцов сосредоточено в старших возрастых группах, клиентов, находящихся в разводе - в возрастной категории 3 и 4, а клиенты, состоящие в гражданском браке и клиенты не состоящие в браке - это в основном клиенты из возрастных групп 1 и 2. Следовательно, возраст и семейный статус - это признаки, характеризующиеся высокой корреляцией, это необходимо будет учесть при создании модели скоринга. 

- Как разные цели кредита влияют на его возврат в срок?

In [88]:
pivot_purpose =df_short.pivot_table(index='purpose_type', columns='debt', values='gender', aggfunc='count')
pivot_purpose ['percent_non_repayment'] = (pivot_purpose[1] / (pivot_purpose[0] +
                                                                          pivot_purpose[1]) * 100).round(1)
pivot_purpose .sort_values('percent_non_repayment')

debt,0,1,percent_non_repayment
purpose_type,,,
недвижимость,9926,776,7.3
свадьба,2129,181,7.8
автомобиль,3863,396,9.3
образование,3602,369,9.3


**Вывод**

Наибольший процент невозвратов в срок у кредитов взятых для целей "образование" и "автомобиль" - по 9,3% , наименьший процент невозвратов у кредитов, взятых под цели "недвижимость" - 7,3%. Возможно это связано с тем, что для приобретения недвижимости берутся существенно большие суммы по сравнению с другими и, соответственно, клиенты подвергаются более жесткой проверке на предмет кредитоспособности.

## Шаг 4. Общий вывод

В ходе выполнения проекта была произведена предобработка обезличенных данных о заемщиках, c целью подготовки этих данных к последующему использованию при создании модели кредитного скорринга. Удалены пропуски; обработаны данные, содержащие некорректные значения и ошибки; устранены явные  и неявные дубли, в т.ч. с использованием метода лемматизации; ряд признаков был категоризирован, некоторые ранее существующие категории укрупнены и преобразованы. На конечном этапе была произведена оценка влияния различных признаков на возврат кредита в срок.  
Итоговая таблица готова к передаче для последующего анализа и моделирования.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.